# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
pip install gmaps


  Created wheel for gmaps: filename=gmaps-0.9.0-py2.py3-none-any.whl size=2076101 sha256=e20a415c7c88f0b35810d8602acf6234df3c5cccf0acd421b56402de61c3d053
  Stored in directory: c:\users\ayala\appdata\local\pip\cache\wheels\30\bd\87\d0d731ee712c34a4d6bace8b1dbf8c8932e22a1625b4a55be4
Successfully built gmaps


In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
api_key = g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# Load Data
Weather_Data = pd.read_csv("Resources/cities.csv")
Weather_Data.head()


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure Gmaps
gmaps.configure(api_key)

# Set Lat and Lng locations
locations = Weather_Data[['Lat','Lng']]

# Humidity 
Humidity = Weather_Data['Humidity']

In [5]:
# Ploting the Heatmap
fig = gmaps.figure()
# Create Heat Layer
Heat_Layer = gmaps.heatmap_layer(locations, weights = Humidity)
# Add Layer
fig.add_layer(Heat_Layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
# Narrow Conditions for Cities
# Wind Speed Less than 10 MPH, Cloudiness = 0, Temperatures between 60 and 70 deegrees
Narrowed_df = Weather_Data.loc[(Weather_Data['Wind Speed'] <=10 ) & (Weather_Data['Cloudiness'] == 0) & (Weather_Data['Max Temp'] >= 70) & (Weather_Data['Max Temp'] <= 80)].dropna()
Narrowed_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
# Hotel Map
Hotel_df = Narrowed_df.loc[:,['City','Country','Lat','Lng']]
# Add Column
Hotel_df['Hotel Name'] = ""
# Show DataFrame
Hotel_df

,City,Country,Lat,Lng,Hotel Name
70,ponta do sol,BR,-20.63,-46.00,
88,cidreira,BR,-30.17,-50.22,
138,morondava,MG,-20.30,44.28,
139,vaini,IN,15.34,74.49,
176,nador,MA,35.17,-2.93,
297,mogok,MM,22.92,96.51,
444,birjand,IR,32.86,59.22,


In [13]:
# Set URL 
url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
# Set Paramenters
params = {'type':'hotel',
          'keyword':'hotel',
          'radius': 5000,
          'key': api_key
}

In [21]:
# Store Results into the DF
for index, row in Hotel_df.iterrows():
    Lat = row['Lat']
    Lng = row['Lng']
    City = row['City']
    
    params['location'] = f'{Lat},{Lng}'
    
    #URL and API
    print(f'Result for Index {index}: {City}')
    Response = requests.get(url, params=params).json()
    
    #Extracting String
    Cities = Response['results']
    
    # Get Closest Hotel 
    
    try:
        print(f'The closest hotel to destination {City} is {Cities[0]["name"]}.')
        Hotel_df.loc[index, 'Hotel Name'] = Cities[0]['name']
        
    # If there is no hotel
    except (KeyError, IndexError):
        print('No Hotel in City')
        
        
    
    


Result for Index 70: ponta do sol
The closest hotel to destination ponta do sol is Pousada Buena Vista.
Result for Index 88: cidreira
The closest hotel to destination cidreira is Hotel Castelo.
Result for Index 138: morondava
The closest hotel to destination morondava is Palissandre Cote Ouest resort & SPA.
Result for Index 139: vaini
No Hotel in City
Result for Index 176: nador
The closest hotel to destination nador is Hotel Marchica Lagoon Resort, Nador Morocco.
Result for Index 297: mogok
The closest hotel to destination mogok is Mount Mogok Hotel.
Result for Index 444: birjand
No Hotel in City


In [22]:
# Display DataFrame with Hotel Names
Hotel_df

,City,Country,Lat,Lng,Hotel Name
70,ponta do sol,BR,-20.63,-46.00,Pousada Buena Vista
88,cidreira,BR,-30.17,-50.22,Hotel Castelo
138,morondava,MG,-20.30,44.28,Palissandre Cote Ouest resort & SPA
139,vaini,IN,15.34,74.49,
176,nador,MA,35.17,-2.93,"Hotel Marchica Lagoon Resort, Nador Morocco"
297,mogok,MM,22.92,96.51,Mount Mogok Hotel
444,birjand,IR,32.86,59.22,هتل پرويز


In [23]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in Hotel_df.iterrows()]
locations = Hotel_df[["Lat", "Lng"]]

In [25]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(markers)

# Display Map

fig

Figure(layout=FigureLayout(height='420px'))